In [1]:
from tqdm import trange

In [2]:
import tensorflow as tf

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [6]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])  # 1 의미 : 흑백?

In [7]:
Y = tf.placeholder(tf.float32, [None, 10])

In [8]:
is_training = tf.placeholder(tf.bool)

In [9]:
L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu) ## X : 입력값, 32: 채널수, [3,3] : 필터 크기

In [11]:
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2]) # [2, 2]: 커널 크기, [2, 2]: 스트라이드 크기

In [12]:
L1 = tf.layers.dropout(L1, 0.7, is_training)

In [13]:
L2 = tf.layers.conv2d(L1, 64, [3,3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.5, is_training)

In [15]:
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.7, is_training)

In [16]:
model = tf.layers.dense(L3, 10, activation=None)

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

In [18]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [26]:
## 학습
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in trange(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28,28,1)
        _, cost_val = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y:batch_ys, is_training:True})
        total_cost += cost_val
    print("Epoch: %04d" % (epoch+1), 'Avg. cost: {:.4f}'.format(total_cost / total_batch))
print("학습 완료")




  0%|                                                                                           | 0/15 [00:00<?, ?it/s]

Epoch: 0001 Avg. cost: 0.0308





  7%|█████▌                                                                             | 1/15 [00:26<06:10, 26.44s/it]

Epoch: 0002 Avg. cost: 0.0215





 13%|███████████                                                                        | 2/15 [00:53<05:44, 26.51s/it]

...

Epoch: 0014 Avg. cost: 0.0046





 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [06:20<00:27, 27.71s/it]

Epoch: 0015 Avg. cost: 0.0044





100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [06:47<00:00, 27.59s/it]




학습 완료


In [27]:
## 테스트 정확도 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도: ", sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1, 28,28,1), Y:mnist.test.labels, is_training:False}))

정확도:  0.9935


In [28]:
sess.close()